### Decorator Factory
A decorator factory is a function that returns a decorator.<br> 
Instead of returning a function (inner in our case), we will return a decorator (named “dec”).<br> 
Lets see this:

In [1]:

from functools import wraps

def inject_variables(context):
    """ Decorator factory. """

    def variable_injector(func):
        """ Decorator. """
        @wraps(func)
        def decorator(*args, **kwargs):
            func_globals = func.__globals__

            # Save copy of any global values that will be replaced.
            saved_values = {key: func_globals[key] for key in context
                                                        if key in func_globals}
            func_globals.update(context)
            try:
                result = func(*args, **kwargs)
            finally:
                func_globals.update(saved_values)  # Restore replaced globals.

            return result

        return decorator

    return variable_injector



namespace = dict(a=5, b=3)

@inject_variables(namespace)
def test():
    print('a:', a)
    print('b:', b)

test()

a: 5
b: 3


In [4]:
from time import perf_counter

def decorator_factory(loops_num):
    def decorating(fn): #wrapped function
        def inner(num):   
            total_elapsed = 0
            for i in range(loops_num):
                start = perf_counter()
                result = fn(num) #wrapped function
                end = perf_counter()
                total_elapsed += end - start
            avg_run_time = total_elapsed/loops_num
            print('result is', result)    
            print('num of loops is', loops_num)
            print('avg time elapsed', avg_run_time)
        return inner
    return decorating

See that here we return both :
- inner function 
- decorating function. 

What we are returning, is a decorator.<br>

This enables the inner function to have access to some additional parameters<br>
(e.g a, b) and use them.

In [5]:
@decorator_factory(500)
def calc_factorial2(num):
    if num < 0:
        raise ValueError('Please use a number not smaller than 0')
    product = 1
    for i in range(num):
        product = product * (i+1)
    return product

calc_factorial2(4)

result is 24
num of loops is 500
avg time elapsed 6.25296000293929e-07


## Class Decorator Factory
Lets now look at an alternative way to generate the same behavior, but with a class.<br>

A class is sometimes easier to make more complicated operations with,<br>

and so its an important tool to have.<br>

Lets do exactly what we did with the decorator factory function, this time with a class.

In [6]:
class Decorator_Factory_Class:
    def __init__(self, num_loops):
        self.num_loops = num_loops
    def __call__(self, fn):
          def inner(num):   
            total_elapsed = 0
            for i in range(self.num_loops):
                start = perf_counter()
                result = fn(num)
                end = perf_counter()
                total_elapsed += end - start
            avg_run_time = total_elapsed/self.num_loops
            print('num of loops is', self.num_loops)
            return result
          return inner

In [7]:
@Decorator_Factory_Class(5)
def calc_factorial2(num):
    if num < 0:
        raise ValueError('Please use a number not smaller than 0')
    product = 1
    for i in range(num):
        product = product * (i+1)
    return product
calc_factorial2(4)

num of loops is 5


24